In [1]:
import os
import csv
import sys
import random
import scipy
import numpy as np
from sets import Set


workspace_path = '/home/irashadow/python_workspace/Kaggle_Comp/Santander/'

/home/irashadow/env_TensorFlow/lib/python2.7/site-packages/ipykernel/__main__.py:7: DeprecationWarning: the sets module is deprecated


In [2]:
month_dict_inv = dict()
month_dict_inv[0] = '2015-01'
month_dict_inv[1] = '2015-02'
month_dict_inv[2] = '2015-03'
month_dict_inv[3] = '2015-04'
month_dict_inv[4] = '2015-05'
month_dict_inv[5] = '2015-06'
month_dict_inv[6] = '2015-07'
month_dict_inv[7] = '2015-08'
month_dict_inv[8] = '2015-09'
month_dict_inv[9] = '2015-10'
month_dict_inv[10] = '2015-11'
month_dict_inv[11] = '2015-12'
month_dict_inv[12] = '2016-01'
month_dict_inv[13] = '2016-02'
month_dict_inv[14] = '2016-03'
month_dict_inv[15] = '2016-04'
month_dict_inv[16] = '2016-05'
month_dict_inv[17] = '2016-06'

In [4]:
cust_2_labels_dict = dict()

pos_case_dict = dict()
neg_case_dict = dict()

for i in range(1,25): pos_case_dict[i] = 0 
for i in range(1,25): neg_case_dict[i] = 0     

train_write_set = [open(workspace_path + '/train/model_comp2/cust_product_1.train', 'w')]
for i in range(2,25):
    train_write_set.append(open(workspace_path + '/train/model_comp2/cust_product_'+str(i)+'.train', 'w'))

check_line_count = 0


feat_data = []
#for date_index_major in [9,11,12,13,15,16]:
for date_index_major in range(5,17):
    target_date = month_dict_inv[date_index_major]
    feat_data.append(open(workspace_path + '/L4_feat_max/4_7_cust_date_nominal_prod_L4_'+str(target_date)+'_Model.feat', 'r'))

    
    
for feat_file in feat_data:
    for line in feat_file:

        line = line.strip()
        check_line_count += 1

        if len(line.split('|')) < 6:

           print line 

        cust_id = line.split('|')[0]
        date_note = line.split('|')[1]
        label_tokens = line.split('|')[2]
        added_product_features = line.split('|')[3]
        product_features = line.split('|')[4]
        nominal_features_pres = line.split('|')[5]
        real_features_pres = line.split('|')[6]

        get_label = np.zeros(30)

        for label_item in label_tokens.split(','):

            if len(label_item) > 0:

                prod_id = int(label_item)
                get_label[prod_id] = 1


        if True:

           for j in range(1,25):

               if get_label[j] == 1:

                    prt_line = '+1' + ' ' 
                    for fi in range(3,10):
                        if len(line.split('|')[fi]) > 0:
                           prt_line += line.split('|')[fi] + ' ' 

                    train_write_set[j-1].write(prt_line[:-1]+'\n') 
                    pos_case_dict[j] += 1 

               elif get_label[j] == 0:

                    prt_line = '-1' + ' ' 
                    for fi in range(3,10):
                        if len(line.split('|')[fi]) > 0:
                           prt_line += line.split('|')[fi] + ' ' 

                    train_write_set[j-1].write(prt_line[:-1]+'\n')  
                    neg_case_dict[j] += 1 
    
for i in range(0,24):
    train_write_set[i].close()
#for i in range(0,23):
    #feat_data[i].close()    
    
#feat_file.close()   
print check_line_count    
#feat_read.close() 

436493


In [5]:
import random



for i in [3,24,19,18,23,22,13,5,7,20,8,9,14,6,15,16,4,17]:
#for i in [3]:
       
    
    train_read = open(workspace_path + '/train/model_comp2/cust_product_'+str(i)+'.train', 'r')
    
    pos_count = 0
    neg_count = 0
    
    print 'sub-model_' + str(i)
    
    for line in train_read:
        
        line = line.strip()
        
        if line[0:2] == '+1':
            
           pos_count += 1
        
        elif line[0:2] == '-1':
            
           neg_count += 1 
        
        
    pos_rate = pos_count/float(pos_count+neg_count)
    neg_rate = neg_count/float(pos_count+neg_count)
    
    print 'Pos. Rate:%.4f'%(pos_rate)
    print 'Neg. Rate:%.4f'%(neg_rate)
    
    train_read = open(workspace_path + '/train/model_comp2/cust_product_'+str(i)+'.train', 'r')
    train_write = open(workspace_path + '/train/model_balance/cust_product_'+str(i)+'.train', 'w')
    
    neg_sampling_count = 0
    
    for line in train_read:
        
        line = line.strip()
        
        if line[0:2] == '+1':
            
           train_write.write(line+'\n')
        
        elif line[0:2] == '-1':
            
           if pos_rate >= 0.2:
            
              sampling_rate = (pos_rate*2)/float(neg_rate)
                
           elif pos_rate >= 0.01:
            
              sampling_rate = (pos_rate*4)/float(neg_rate)     
                
           else:
            
              sampling_rate = (pos_rate*20)/float(neg_rate)
            
        
           if random.random() < sampling_rate:
            
              neg_sampling_count += 1
              train_write.write(line+'\n')  
        
        
    neg_sampling_rate = neg_sampling_count/float(pos_count+neg_count)
    
    print 'neg_sampling_rate:%.4f'%(neg_sampling_rate)
    
    train_read.close()
    train_write.close()
            
            

sub-model_3
Pos. Rate:0.3250
Neg. Rate:0.6750
neg_sampling_rate:0.6500
sub-model_24
Pos. Rate:0.2779
Neg. Rate:0.7221
neg_sampling_rate:0.5549
sub-model_19
Pos. Rate:0.1195
Neg. Rate:0.8805
neg_sampling_rate:0.4777
sub-model_18
Pos. Rate:0.0175
Neg. Rate:0.9825
neg_sampling_rate:0.0708
sub-model_23
Pos. Rate:0.1520
Neg. Rate:0.8480
neg_sampling_rate:0.6078
sub-model_22
Pos. Rate:0.1336
Neg. Rate:0.8664
neg_sampling_rate:0.5339
sub-model_13
Pos. Rate:0.0470
Neg. Rate:0.9530
neg_sampling_rate:0.1876
sub-model_5
Pos. Rate:0.0695
Neg. Rate:0.9305
neg_sampling_rate:0.2775
sub-model_7
Pos. Rate:0.0189
Neg. Rate:0.9811
neg_sampling_rate:0.0761
sub-model_20
Pos. Rate:0.0092
Neg. Rate:0.9908
neg_sampling_rate:0.1851
sub-model_8
Pos. Rate:0.0064
Neg. Rate:0.9936
neg_sampling_rate:0.1283
sub-model_9
Pos. Rate:0.0040
Neg. Rate:0.9960
neg_sampling_rate:0.0800
sub-model_14
Pos. Rate:0.0041
Neg. Rate:0.9959
neg_sampling_rate:0.0826
sub-model_6
Pos. Rate:0.0023
Neg. Rate:0.9977
neg_sampling_rate:0.045

In [7]:
for model_no in [3,24,19,18,23,22,13,5,7,20,8,9,14,6,15,16,4,17]:

    train_data_path = './train/model_balance/cust_product_'+str(model_no)+'.train'
    #train_data_path = './train/model_exp/cust_product_'+str(model_no)+'.train'
    model_path = '../liblinear-2.1/model_save/cust_product_'+str(model_no)+'.model'
    Ein_path = '../liblinear-2.1/output/cust_product_'+str(model_no)+'.Ein'

    train_command = '../liblinear-2.1/./train -s 6 -c 2'
    predi_command = '../liblinear-2.1/./predict'


    #print os.system("pwd");
    os.system(train_command +' '+ train_data_path +' '+ model_path);
    os.system(predi_command +' '+ train_data_path +' '+ model_path +' '+ Ein_path);
    
